In [1]:
#Import dependencies
import pandas as pd
import csv
import numpy

In [2]:
#File to Load
school_data_path = "Resources/schools_complete.csv"
students_data_path = "Resources/students_complete.csv"

In [3]:
# DataFrames creations 
school_data = pd.read_csv(school_data_path)
students_data = pd.read_csv(students_data_path)
school_data = pd.DataFrame(school_data)
students_data = pd.DataFrame(students_data)

In [4]:
# Schools data overview
print(school_data.shape)
school_data

(15, 5)


,School ID,school_name,type,size,budget
0,0,Huang High School,District,2917,1910635
1,1,Figueroa High School,District,2949,1884411
2,2,Shelton High School,Charter,1761,1056600
3,3,Hernandez High School,District,4635,3022020
4,4,Griffin High School,Charter,1468,917500
5,5,Wilson High School,Charter,2283,1319574
6,6,Cabrera High School,Charter,1858,1081356
7,7,Bailey High School,District,4976,3124928
8,8,Holden High School,Charter,427,248087
9,9,Pena High School,Charter,962,585858


In [5]:
# Students data overview
print(students_data.shape)
students_data

(39170, 7)


,Student ID,student_name,gender,grade,school_name,reading_score,math_score
0,0,Paul Bradley,M,9th,Huang High School,66,79
1,1,Victor Smith,M,12th,Huang High School,94,61
2,2,Kevin Rodriguez,M,12th,Huang High School,90,60
3,3,Dr. Richard Scott,M,12th,Huang High School,67,58
4,4,Bonnie Ray,F,9th,Huang High School,97,84
...,...,...,...,...,...,...,...
39165,39165,Donna Howard,F,12th,Thomas High School,99,90
39166,39166,Dawn Bell,F,10th,Thomas High School,95,70
39167,39167,Rebecca Tanner,F,9th,Thomas High School,73,84
39168,39168,Desiree Kidd,F,10th,Thomas High School,99,90


In [6]:
students_data.loc[[0,10],["student_name","gender"]]

,student_name,gender
0,Paul Bradley,M
10,Andrew Alexander,M


In [51]:
# District Summary 
District_summary = pd.DataFrame()
District_summary["total_schools"] = [len(school_data)]
District_summary["total_students"] = [len(students_data)]
District_summary["total_students"] = District_summary["total_students"].map("{:,}".format)
District_summary["total_budget"] = school_data["budget"].sum(axis=0)
District_summary["total_budget"] = District_summary["total_budget"].map("${:,.2f}".format)
District_summary["average_math"] = students_data["math_score"].mean()
District_summary["average_reading"] = students_data["reading_score"].mean()
District_summary["% Passing Math"] = ((students_data[students_data["math_score"]>=70].count().math_score)*100)/len(students_data)
District_summary["% Passing Reading"] = ((students_data[students_data["reading_score"]>=70].count().reading_score)*100)/len(students_data)
District_summary["% Overall Passing"] = students_data[(students_data["math_score"]>=70) & (students_data["reading_score"]>=70)].count().student_name
District_summary["% Overall Passing"] = (District_summary["% Overall Passing"]*100)/len(students_data)

District_summary["% Passing Math"] = District_summary["% Passing Math"].map("{:,.3f}%".format)
District_summary["% Passing Reading"] = District_summary["% Passing Reading"].map("{:,.3f}%".format)
District_summary["% Overall Passing"] = District_summary["% Overall Passing"].map("{:,.3f}%".format)

District_summary

,total_schools,total_students,total_budget,average_math,average_reading,% Passing Math,% Passing Reading,% Overall Passing
0,15,"39,170","$24,649,428.00",78.985371,81.87784,74.981%,85.805%,65.172%


In [8]:
# School Summary
# Merging both DataFrames to complete the School Summary
Final_data = pd.merge(school_data, students_data, on ="school_name")
Final_data.head()

,School ID,school_name,type,size,budget,Student ID,student_name,gender,grade,reading_score,math_score
0,0,Huang High School,District,2917,1910635,0,Paul Bradley,M,9th,66,79
1,0,Huang High School,District,2917,1910635,1,Victor Smith,M,12th,94,61
2,0,Huang High School,District,2917,1910635,2,Kevin Rodriguez,M,12th,90,60
3,0,Huang High School,District,2917,1910635,3,Dr. Richard Scott,M,12th,67,58
4,0,Huang High School,District,2917,1910635,4,Bonnie Ray,F,9th,97,84


In [69]:
School_Summary = pd.DataFrame()
School_Summary = Final_data.loc[:,["school_name","type", "budget"]]
School_Summary = School_Summary.drop_duplicates()
School_Summary = School_Summary.set_index(["school_name"])
School_Summary["Total Students"] = Final_data.groupby(["school_name"])["School ID"].count()
School_Summary["Per Student Budget"] = School_Summary["budget"]/School_Summary["Total Students"]
School_Summary["Average Math Score"] = Final_data.groupby(["school_name"])["math_score"].mean()
School_Summary["Average Reading Score"] = Final_data.groupby(["school_name"])["reading_score"].mean()
School_Summary["% Passing Math"] = ((Final_data.loc[Final_data["math_score"]>=70,["school_name","math_score"]]).groupby(["school_name"])["math_score"].count()*100)/School_Summary["Total Students"]
School_Summary["% Passing Reading"] = ((Final_data.loc[Final_data["reading_score"]>=70,["school_name","reading_score"]]).groupby(["school_name"])["reading_score"].count()*100)/School_Summary["Total Students"]
School_Summary["% Overall Passing"] = ((Final_data[(Final_data["math_score"]>=70) & (Final_data["reading_score"]>=70)].groupby(["school_name"])["school_name"].count())*100)/School_Summary["Total Students"]
School_Summary["budget"] = School_Summary["budget"].map("${:,.2f}".format)
School_Summary["Per Student Budget"] = School_Summary["Per Student Budget"].map("${:,.2f}".format)
School_Summary["% Passing Math"] = School_Summary["% Passing Math"].map("{:,.3f}%".format)
School_Summary["% Passing Reading"] = School_Summary["% Passing Reading"].map("{:,.3f}%".format)
School_Summary["% Overall Passing"] = School_Summary["% Overall Passing"].map("{:,.3f}%".format)
School_Summary = School_Summary.rename(columns = {"type":"School Type","budget":"Total School Budget"})
School_Summary = School_Summary.sort_values(["school_name"])
School_Summary = School_Summary[["School Type","Total Students","Total School Budget","Per Student Budget","Average Math Score","Average Reading Score", "% Passing Math", "% Passing Reading", "% Overall Passing"]]
School_Summary


,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Bailey High School,District,4976,"$3,124,928.00",$628.00,77.048432,81.033963,66.680%,81.933%,54.642%
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133%,97.040%,91.335%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988%,80.739%,53.204%
Ford High School,District,2739,"$1,763,916.00",$644.00,77.102592,80.746258,68.310%,79.299%,54.290%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392%,97.139%,90.599%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.753%,80.863%,53.528%
Holden High School,Charter,427,"$248,087.00",$581.00,83.803279,83.814988,92.506%,96.253%,89.227%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.684%,81.316%,53.514%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.058%,81.222%,53.539%


In [73]:
# Top Performing Schools (By % Overall Passing)
Top_Schools = pd.DataFrame()
Top_Schools = School_Summary.sort_values(["% Overall Passing"], ascending = False)
Top_Schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Cabrera High School,Charter,1858,"$1,081,356.00",$582.00,83.061895,83.975780,94.133%,97.040%,91.335%
Thomas High School,Charter,1635,"$1,043,130.00",$638.00,83.418349,83.848930,93.272%,97.309%,90.948%
Griffin High School,Charter,1468,"$917,500.00",$625.00,83.351499,83.816757,93.392%,97.139%,90.599%
Wilson High School,Charter,2283,"$1,319,574.00",$578.00,83.274201,83.989488,93.868%,96.540%,90.583%
Pena High School,Charter,962,"$585,858.00",$609.00,83.839917,84.044699,94.595%,95.946%,90.541%


In [74]:
Bottom_Schools = pd.DataFrame()
Bottom_Schools = School_Summary.sort_values(["% Overall Passing"])
Bottom_Schools.head(5)

,School Type,Total Students,Total School Budget,Per Student Budget,Average Math Score,Average Reading Score,% Passing Math,% Passing Reading,% Overall Passing
school_name,,,,,,,,,
Rodriguez High School,District,3999,"$2,547,363.00",$637.00,76.842711,80.744686,66.367%,80.220%,52.988%
Figueroa High School,District,2949,"$1,884,411.00",$639.00,76.711767,81.158020,65.988%,80.739%,53.204%
Huang High School,District,2917,"$1,910,635.00",$655.00,76.629414,81.182722,65.684%,81.316%,53.514%
Hernandez High School,District,4635,"$3,022,020.00",$652.00,77.289752,80.934412,66.753%,80.863%,53.528%
Johnson High School,District,4761,"$3,094,650.00",$650.00,77.072464,80.966394,66.058%,81.222%,53.539%
